<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Make-data" data-toc-modified-id="Make-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Make data</a></span><ul class="toc-item"><li><span><a href="#Make-network" data-toc-modified-id="Make-network-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Make network</a></span><ul class="toc-item"><li><span><a href="#Make-cases" data-toc-modified-id="Make-cases-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Make cases</a></span></li></ul></li></ul></li><li><span><a href="#Compute-gradient" data-toc-modified-id="Compute-gradient-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Compute gradient</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Tasks" data-toc-modified-id="Tasks-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Tasks</a></span></li></ul></li></ul></li><li><span><a href="#Comparing-jacobians" data-toc-modified-id="Comparing-jacobians-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Comparing jacobians</a></span></li></ul></div>

In [1]:
using Pkg; Pkg.activate()

using Distributions, Random
using LightGraphs
using Convex, ECOS
using Plots

using Revise
using CarbonNetworks

ECOS_QUIET = () -> ECOS.Optimizer(verbose=false)

  Activating environment at `~/.julia/environments/v1.6/Project.toml`
┌ Info: Precompiling CarbonNetworks [d5dced0a-5569-40db-8865-1dbf132a76f2]
└ @ Base loading.jl:1317
┌ Warning: Package CarbonNetworks does not have SparseArrays in its dependencies:
│ - If you have CarbonNetworks checked out for development and have
│   added SparseArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with CarbonNetworks
└ Loading SparseArrays into CarbonNetworks from project dependency, future warnings for CarbonNetworks are suppressed.


#5 (generic function with 1 method)

In [2]:
using LinearAlgebra

# Make data

## Make network

In [152]:
Random.seed!(2)
n = 3

# Make graph
G = watts_strogatz(n, 2, 0.2)

# Convert to incidence matrix
A = incidence_matrix(G, oriented=true)
m = size(A, 2)

# Generate costs
f = rand(Exponential(5), n) .+ 2

# Generate generation and flow capacities
gmax = rand(Gamma(5.0, 5.0), n)
pmax = rand(Gamma(1.0, 1.0), m);

In [154]:
B = I;

### Make cases

In [155]:
Random.seed!(3)

num_cases = 500

cases = []
for _ in 1:num_cases
    d = rand(Uniform(0.5, 1.0), n) .* gmax
    
    opf = PowerManagementProblem(f, d, pmax, gmax, A)
    solve!(opf, ECOS_QUIET)
    
    push!(cases, (d=d, g=evaluate(opf.g)))
end

train_cases = cases[1:400]
test_cases = cases[401:end];

# Compute gradient

We want to solve 

$$\text{minimize}_\theta\quad L(g^*(\theta)) + \frac{\lambda}{2} \| f \|_2^2$$
    
where $L(g) = \frac{1}{2} \| g - g_{\mathrm{true}} \|_2^2$ and $g^*(\theta)$ is the OPF mapping. This gradient of the loss is $\nabla L(g) = g - g_{\mathrm{true}}$.

### Tasks

- [ ] Speed up algorithm via warm starting
- [ ] Tune regularization parameters
- [ ] Verify that LMPs are similar to true LMPs (since that is what matters)

In [ ]:
# function loss_and_grad(f̂, cases)
#     L = 0.0
#     df = zeros(n)
    
#     T = length(cases)
#     _∇L = zeros(kkt_dims(n, m))

#     for case in cases
#         params = (f̂, case.d, pmax, gmax, A)
#         opf = PowerManagementProblem(params...)
#         solve!(opf, ECOS_QUIET)
#         ĝ = evaluate(opf.g)
        
#         L += (1/2) * norm(ĝ - case.g)^2 / T
        
#         _∇L[1:n] .+= ĝ - case.g
#         df += sensitivity_price(opf, _∇L, params) / T
#     end
    
#     return L, df
# end

# function stochastic_loss_and_grad(f̂, cases, sample)
#     return loss_and_grad(f̂, cases[sample])
# end

In [ ]:
Random.seed!(4)

# Algorithm parameters
step_size = 5.0
max_iter = 200
batch_size = 2
test_batch_size = 2
λ = 0.0
lag = 100

# Initialize estimated costs
f̂ = rand(Exponential(5), n) .+ 2

train_loss_hist = []
test_loss_hist = []
grad_hist = []
acc_hist = []

@time for iter in 1:max_iter
    # Evaluate loss and gradient
    sample = rand(1:length(train_cases), batch_size)
    L, df = stochastic_loss_and_grad(f̂, B, train_cases, pmax, gmax, A, sample)
    
    push!(train_loss_hist, L)
    push!(grad_hist, norm(df))
    push!(acc_hist, abs(f̂'f) / (norm(f̂) * norm(f)))
    
    # Compute test loss
    sample = rand(1:length(test_cases), test_batch_size)
    L_test, _ = stochastic_loss_and_grad(f̂, B, train_cases, pmax, gmax, A, sample)
    push!(test_loss_hist, L_test)
    
    # Take projected gradient step
    η = step_size #* (lag / (lag + sqrt(iter)))
    Δ = df + λ*f
    #@show norm(Δ)
    f̂ = max.(f̂ - η*(Δ/norm(Δ)), 0)
end
println("Completed $(max_iter) iterations.")

In [ ]:
theme(:default, lw=4, label=nothing)

plot(
    plot(train_loss_hist, ylabel="train loss"),
    plot(test_loss_hist, ylabel="test loss"),
    plot(acc_hist, ylabel="angle(f, f̂)"),
    layout=(3, 1)
)

In [ ]:
@show norm(f̂)
@show norm(f)
plot(bar(f̂ / norm(f̂)), bar(f/norm(f)), layout=(2, 1))

In [ ]:
merit_order = sortperm(f)
est_merit_order = sortperm(f̂)

---


In [ ]:
Lg, Lp = size(A)

In [ ]:
λpu = rand(Lp);
λpl = rand(Lp);
λgu = rand(Lg);
λgl = rand(Lg);

In [ ]:
K11 = zeros(Lp + Lg, Lp+Lg);
K22 = Diagonal(vcat(λpu, λpl, λgu, λgl)); #wrong - you have to take the values
K33 = zeros(Lg);

K12 = vcat(
    hcat(Matrix(I, Lp, Lp), -Matrix(I, Lp, Lp), zeros(Lp, 2*Lg)), 
    hcat(zeros(Lg, 2*Lp), Matrix(I, Lg, Lg), -Matrix(I, Lg, Lg))
    );


K13 = vcat(Matrix(I, Lg, Lg), -A');

K21 = hcat(
    vcat(Diagonal(λpu), -Diagonal(λpl), zeros(2*Lg, Lp)), 
    vcat(zeros(2*Lp, Lg), Diagonal(λgu), -Diagonal(λgl))
    );

K = vcat(
    hcat(K11, K12, K13), 
    hcat(K21, K22, zeros(2*(Lp+Lg), Lg)), 
    hcat(K13', zeros(Lg, 2*(Lp+Lg)+Lg))
    );

# Comparing jacobians

In [156]:
f̂ = rand(Exponential(5), n) .+ 2
case = cases[1]
params = (f̂, case.d, pmax, gmax, A)
opf = PowerManagementProblem(params...)
solve!(opf, () -> ECOS.Optimizer(verbose=false), verbose=false)
ĝ = evaluate(opf.g);

In [242]:
J1, J2 = CarbonNetworks.compare_jacobians(opf, params);

In [243]:
J1 = J1';

In [244]:
norm(J1 - J2)

32.622906046580304

In [234]:
n, m = size(A)

(3, 3)

In [206]:
#K12
JJ1 = J1[1:(m+n), m+n+1:3(m+n)]
JJ2 = J2[1:(m+n), m+n+1:3(m+n)];

In [210]:
#K13 ==> not symmetric?
JJ1 = J1[1:(m+n), 3(m+n)+1:end]
JJ2 = J2[1:(m+n), 3(m+n)+1:end];

In [235]:
#K21
JJ1 = J1[(m+n+1):3(m+n), 1:(m+n)]
JJ2 = J2[(m+n+1):3(m+n), 1:(m+n)];

In [245]:
#K22
JJ1 = J1[(m+n+1):3(m+n), (m+n+1):3(m+n)]
JJ2 = J2[(m+n+1):3(m+n), (m+n+1):3(m+n)];

A few questions about the structure of the Jacobian: 
- the central block is not diagonal? 
- the corner blocks are anti-symmetric?
- what is the order in which the derivatives are computed? (i.e. the ordering in the blocks?)

In [250]:
JJ1[1:5, 1:5]

5×5 Matrix{Float64}:
 0.0          0.0         0.0  -0.181415   0.0
 0.0          0.0         0.0   0.0       -1.79048
 0.0          0.0         0.0   0.0        0.0
 2.00502e-8   0.0         0.0   0.0        0.0
 0.0         -2.03516e-8  0.0   0.0        0.0

In [256]:
diag(JJ1)

12-element Vector{Float64}:
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
 -13.930015241517067
 -37.38763638200595
 -32.18299232299253
  -0.35941116086532077
  -0.2994371120439894
  -4.425854530103479

In [251]:
JJ2[1:5, 1:5]

5×5 SparseArrays.SparseMatrixCSC{Float64, Int64} with 5 stored entries:
 -2.00502e-8   ⋅           ⋅            ⋅          ⋅ 
   ⋅          2.03516e-8   ⋅            ⋅          ⋅ 
   ⋅           ⋅          1.62416e-8    ⋅          ⋅ 
   ⋅           ⋅           ⋅          -0.181415    ⋅ 
   ⋅           ⋅           ⋅            ⋅        -1.79048

In [211]:
JJ1-JJ2

6×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [208]:
JJ2

6×12 SparseArrays.SparseMatrixCSC{Float64, Int64} with 12 stored entries:
   ⋅     ⋅     ⋅    ⋅    ⋅    ⋅   -1.0    ⋅     ⋅   1.0   ⋅    ⋅ 
   ⋅     ⋅     ⋅    ⋅    ⋅    ⋅     ⋅   -1.0    ⋅    ⋅   1.0   ⋅ 
   ⋅     ⋅     ⋅    ⋅    ⋅    ⋅     ⋅     ⋅   -1.0   ⋅    ⋅   1.0
 -1.0    ⋅     ⋅   1.0   ⋅    ⋅     ⋅     ⋅     ⋅    ⋅    ⋅    ⋅ 
   ⋅   -1.0    ⋅    ⋅   1.0   ⋅     ⋅     ⋅     ⋅    ⋅    ⋅    ⋅ 
   ⋅     ⋅   -1.0   ⋅    ⋅   1.0    ⋅     ⋅     ⋅    ⋅    ⋅    ⋅ 

In [186]:
maximum(broadcast(abs, JJ1-JJ2))

0.0

In [187]:
JJ1-JJ2

6×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0